In [1]:
import sys
dirname = '/Users/m.daffarobani/Documents/personal_research/smt'
if dirname not in sys.path:
    sys.path.append(dirname)

from scipy import linalg
from smt.utils.misc import compute_rms_error

from smt.problems import Sphere, NdimRobotArm, Rosenbrock
from smt.sampling_methods import LHS
from smt.surrogate_models import LS, QP, KPLS, KRG, KPLSK, GEKPLS, MGP

#to ignore warning messages
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import scipy.interpolate

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
plot_status = True

In [2]:
import numpy as np
from scipy.stats.mstats import mquantiles

def _grid_from_x(X, percentiles, is_categorical, grid_resolution):
    values = []
    # Numeric features
    for i in range(X.shape[1]):
        emp_percentiles = mquantiles(X)
        
def cartesian(arrays, out=None):
    """Generate a cartesian product of input arrays.

    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray of shape (M, len(arrays)), default=None
        Array to place the cartesian product in.

    Returns
    -------
    out : ndarray of shape (M, len(arrays))
        Array containing the cartesian products formed of input arrays.
        If not provided, the `dtype` of the output array is set to the most
        permissive `dtype` of the input arrays, according to NumPy type
        promotion.

        .. versionadded:: 1.2
           Add support for arrays of different types.

    Notes
    -----
    This function may not be used on more than 32 arrays
    because the underlying numpy functions do not support it.

    Examples
    --------
    >>> from sklearn.utils.extmath import cartesian
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """
    arrays = [np.asarray(x) for x in arrays]
    shape = (len(x) for x in arrays)

    ix = np.indices(shape)
    ix = ix.reshape(len(arrays), -1).T

    if out is None:
        dtype = np.result_type(*arrays)  # find the most permissive dtype
        out = np.empty_like(ix, dtype=dtype)

    for n, arr in enumerate(arrays):
        out[:, n] = arrays[n][ix[:, n]]

    return out

In [3]:
ndim = 2
ndoe = 20 #int(10*ndim)
fun = Rosenbrock(ndim=ndim)
sampling = LHS(xlimits=fun.xlimits, criterion='ese', random_state=1)
xt = sampling(ndoe)

In [11]:
xt

In [4]:
percentiles = (0.05, 0.95)
is_categorical = None
grid_resolution = 100

X = xt
values = []
for i in range(X.shape[1]):
    emp_percentiles = mquantiles(X[:, i], prob=percentiles, axis=0)
    if np.allclose(emp_percentiles[0], emp_percentiles[1]):
        raise ValueError(
            "percentiles are too close to each other, "
            "unable to build the grid. Please choose percentiles "
            "that are further apart."
        )
    axis = np.linspace(
        emp_percentiles[0],
        emp_percentiles[1],
        num=grid_resolution,
        endpoint=True)
    values.append(axis)

In [6]:
grid = cartesian(values)

In [7]:
grid

array([[-1.86878202, -1.80821044],
       [-1.86878202, -1.77100673],
       [-1.86878202, -1.73380303],
       ...,
       [ 1.77645109,  1.80054892],
       [ 1.77645109,  1.83775263],
       [ 1.77645109,  1.87495633]])

In [ ]:
np.quantile(X[:,0], 0.95)

1.740618059032943

In [19]:
features = [0, 1, (0, 1), [0, 1], [2]]
for i, feature in enumerate(features):
    if type(feature) in [tuple, list]:
        if len(feature) == 1:
            features[i] = feature[0]
        elif len(feature) == 2:
            features[i] = tuple(feature)
            raise ValueError("Interaction of features hasn't been developed")
        else:
            raise ValueError("Interaction features can't be more than two.")
features

ValueError: Interaction of features hasn't been developed

In [16]:
features

[0, 1, (0, 1), (0, 1)]

In [12]:
features[3] = tuple(features[3])

In [13]:
features

[0, 1, (0, 1), (0, 1)]

In [21]:
X[(0, 1), :]

array([[ 1.53837542, -0.62437651],
       [ 1.73730019,  1.08422153]])

In [23]:
type(1) is int

True